<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/N_Gram_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#N-gram language models

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

## Load data

In [4]:
%%capture
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz

In [5]:
data = pd.read_json("./arxivData.json")

## EDA

In [6]:
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
4083,"[{'name': 'Alessandro Giusti'}, {'name': 'Dan ...",7,1302.1700v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Deep Neural Networks now excel at image classi...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Fast Image Scanning with Deep Max-Pooling Conv...,2013
3714,"[{'name': 'Murat Kocaoglu'}, {'name': 'Christo...",6,1709.02023v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We propose an adversarial training procedure f...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",CausalGAN: Learning Causal Implicit Generative...,2017
100,"[{'name': 'Peter Wittek'}, {'name': 'Sándor Da...",5,1502.01753v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",2,Based on the Aristotelian concept of potential...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Monitoring Term Drift Based on Semantic Consis...,2015
7615,"[{'name': 'Chao Shang'}, {'name': 'Qinqing Liu...",14,1802.04944v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Graph convolutional network (GCN) is generaliz...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Edge Attention-based Multi-Relational Graph Co...,2018
23230,"[{'name': 'Y. Babykalpana'}, {'name': 'K. Than...",23,1005.4216v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Maps are used to describe far-off places . It ...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Classification of LULC Change Detection using ...,2010


In [7]:
data.shape

(41000, 9)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41000 entries, 0 to 40999
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   41000 non-null  object
 1   day      41000 non-null  int64 
 2   id       41000 non-null  object
 3   link     41000 non-null  object
 4   month    41000 non-null  int64 
 5   summary  41000 non-null  object
 6   tag      41000 non-null  object
 7   title    41000 non-null  object
 8   year     41000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 2.8+ MB


In [17]:
for col in data.columns:
  print('Unique:', col, data[col].nunique())
  if data[col].dtype == 'int64':
    print(f"{'_'*5}", col)
    display(data[col].describe())
  print()

Unique: author 36157

Unique: day 31
_____ day


count    41000.000000
mean        15.833293
std          8.699371
min          1.000000
25%          8.000000
50%         16.000000
75%         23.000000
max         31.000000
Name: day, dtype: float64


Unique: id 41000

Unique: link 41000

Unique: month 12
_____ month


count    41000.000000
mean         6.547488
std          3.525254
min          1.000000
25%          3.000000
50%          6.000000
75%         10.000000
max         12.000000
Name: month, dtype: float64


Unique: summary 40967

Unique: tag 5874

Unique: title 40955

Unique: year 26
_____ year


count    41000.000000
mean      2014.484073
std          3.685414
min       1993.000000
25%       2013.000000
50%       2016.000000
75%       2017.000000
max       2018.000000
Name: year, dtype: float64

### Duplicates

In [28]:
data[data.duplicated(subset=['title', 'summary'], keep='first')]

,author,day,id,link,month,summary,tag,title,year
10066,"[{'name': 'Xinghao Pan'}, {'name': 'Jianmin Ch...",19,1702.05800v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Distributed training of deep learning models o...,"[{'term': 'cs.DC', 'scheme': 'http://arxiv.org...",Revisiting Distributed Synchronous SGD,2017
10261,[{'name': 'Daniil Ryabko'}],5,1005.0826v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,"The problem of clustering is considered, for t...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Clustering processes,2010
10824,[{'name': 'David Wipf'}],9,1408.2054v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,In many applications that require matrix solut...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Non-Convex Rank Minimization via an Empirical ...,2014
10826,"[{'name': 'Jie Chen'}, {'name': 'Nannan Cao'},...",9,1408.2060v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,Gaussian processes (GP) are Bayesian non-param...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Parallel Gaussian Process Regression with Low-...,2014
10844,"[{'name': 'Maren Mahsereci'}, {'name': 'Philip...",10,1502.02846v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,"In deterministic optimization, line searches a...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Probabilistic Line Searches for Stochastic Opt...,2015
10944,"[{'name': 'Steffen Lauritzen'}, {'name': 'Nico...",5,1211.0806v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,"Discussion of ""Latent variable graphical model...","[{'term': 'math.ST', 'scheme': 'http://arxiv.o...",Discussion: Latent variable graphical model se...,2012
10945,[{'name': 'Martin J. Wainwright'}],5,1211.0808v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,"Discussion of ""Latent variable graphical model...","[{'term': 'math.ST', 'scheme': 'http://arxiv.o...",Discussion: Latent variable graphical model se...,2012
10946,"[{'name': 'Emmanuel J. Candés'}, {'name': 'Mah...",5,1211.0817v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,"Discussion of ""Latent variable graphical model...","[{'term': 'math.ST', 'scheme': 'http://arxiv.o...",Discussion: Latent variable graphical model se...,2012
15413,"[{'name': 'Daniel Fried'}, {'name': 'Kevin Duh'}]",18,1412.5836v3,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,We investigate the hypothesis that word repres...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Incorporating Both Distributional and Relation...,2014
20087,"[{'name': 'Peter D. Grunwald'}, {'name': 'Jose...",27,1407.7188v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,It is commonly-accepted wisdom that more infor...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",When Ignorance is Bliss,2014


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['day'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['month'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['year'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_3, *['day'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_5, *['year'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(_df_6, *[[['day', 'month'], ['month', 'year']]], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_7, *['year', 'day', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_8, *['year', 'month', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_9, *['year', 'count()', None], **{})
chart

## Graphs

In [29]:
data

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017
...,...,...,...,...,...,...,...,...,...
40995,"[{'name': 'Vitaly Feldman'}, {'name': 'Pravesh...",18,1404.4702v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We study the complexity of learning and approx...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Nearly Tight Bounds on $\ell_1$ Approximation ...,2014
40996,"[{'name': 'Orly Avner'}, {'name': 'Shie Mannor'}]",22,1404.5421v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,We consider the problem of multiple users targ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Concurrent bandits and cognitive radio networks,2014
40997,"[{'name': 'Ran Zhao'}, {'name': 'Deanna Needel...",22,1404.5899v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,"In this paper, we compare and analyze clusteri...","[{'term': 'math.NA', 'scheme': 'http://arxiv.o...",A Comparison of Clustering and Missing Data Me...,2014
40998,"[{'name': 'Zongyan Huang'}, {'name': 'Matthew ...",25,1404.6369v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",4,Cylindrical algebraic decomposition(CAD) is a ...,"[{'term': 'cs.SC', 'scheme': 'http://arxiv.org...",Applying machine learning to the problem of ch...,2014


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(data, *['day'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(data, *['month'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(data, *['year'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(data, *['day'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(data, *['year'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(data, *[[['day', 'month'], ['month', 'year']]], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(data, *['year', 'day', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(data, *['year', 'month', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(data, *['year', 'count()', None], **{})
chart

### Assemble lines: concatenate title and description

In [16]:
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace('', ' '), axis=1).tolist()
sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ;  T h i s   p a p e r   h a s   b e e n   r e t r a c t e d . ',
 'What Does Artificial Life Tell Us About Death? ;  S h o r t   p h i l o s o p h i c a l   e s s a y ',
 "Serious Flaws in Korf et al.'s Analysis on Time Complexity of A* ;  T h i s   p a p e r   h a s   b e e n   w i t h d r a w n . "]